# attempt notebook with API-first structure
In the previous attempt (ocr-test.ipynb), I got all the pieces working, but the code was difficult to read, and made moving to a relation database difficult. I'll prototype an "API-first notebook" to see if a that would be a better structure to bridge the data/backend/frontend.

In [ ]:
# define "API" first, where each function can be replaced by a simple API call

#
# *** IN PSEUDO CODE ***
#

# upload images
def uploadImage(rawFile):
    # write to temporary storage 
    #     -- or keep in case crop doesn't do a good job, which already happened once but then need table for rawFiles too
    return rawFile1TempUrl # assuming passing the URL to another function if don't care for storing it
# crop/convert/postprocess images
def postProcessImages(rawFiles):
    # write to storage
    # write to postProcessedFiles table
    return [postProcessedFile1.Id, postProcessedFile2.Id, postProcessedFile3.Id] # could return success/failure, but prefer to return IDs for reference, might want to return objects depending on how it'll be used later
# ocr images to create receiptTexts
def processImage(postProcessedFile):
    # contains data for boundingbox, text
    # contains reference to filename
    return [receiptText1.Id, receiptText2.Id, receiptText3.Id]
# create receipt
def createReceipt(receiptTexts):
    # contains refrence to receiptTexts, filenames via receiptTexts
    # write to receipts table
    return receipt1.Id

# define/import referenceItems
def createReferenceItem(name, quantity, unitOfMeasure, price, pricePerWeight, referenceUrl):
    # contains data for name, quantity, unitOfMeasure, price, pricePerWeight, referenceUrl
    # write to referenceItems table
    return referenceItem1.Id
# define/import search querries tied to referenceItems
def createEligibleProduct(productName="Schar Gluten Free Hot Dog Buns", referenceItem="Hot dog buns"):
    # contains data for productName, referenceItem
    # write to eligibleProducts table
    return searchQuerry1.Id

# parse receiptTexts against possible product names to find and create eligible eligibleExpenses (description, amount, date, searchQuerry, referenceItem)
def checkForEligibleProductName(receiptTextId):
    def checkProductName(receiptTextId):
        # check if text in receiptText is in eligibleProducts
        return True
    def lookForProductPrice(receiptTextId):
        # bunch of magic here
        return price
    return checkProductName(receiptTextId), lookForProductPrice(receiptTextId)
def createEligibleExpense(description, amount, date, searchQuerry, referenceItemId):
    # contains data for description, amount, date, searchQuerry, referenceItem
    # write to eligibleExpenses table
    return eligibleExpense1.Id
def parseEligibleExpenses(receiptId):
    # requires eligibleProducts table
    # can access receiptTexts via receiptId
    # (?) not sure if a single call is a good way to do it, i.e. maybe a way to bulk process that's better
    #     though a this call will still be necessary at some point, maybe it's a matter of doing a good job
    #     at cleaning the data before firing off individual search querries
    for receiptTextId in receipt[receiptId].receiptTexts:
        if checkForEligibleProductName(receiptTextId):
            createEligibleExpense(priceEach, quantity, receiptTextId, referenceItemId)
    return [eligibleExpense1.Id, eligibleExpense2.Id, eligibleExpense3.Id]